In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random
import csv

path = "/Users/apple/Desktop/CSE258/HW3/assignment1/train_Interactions.csv.gz"
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

dataset = []
allRatings = []
for user, book, r in readCSV(path):
    r = int(r)
    dataset.append([user, book, r])

train = dataset[:190000]
validR = dataset[190000:]

userBooks = defaultdict(set)
bookUsers = defaultdict(set)
userRatings = defaultdict(set)
bookRatings = defaultdict(set)
for item in train:
    u, b, r = item[0], item[1], item[2]
    userBooks[u].add(b)
    bookUsers[b].add(u)
    userRatings[u].add(r)
    bookRatings[u].add(r)



# def prediction(user, item):
#     if user not in userBiases:
#         return alpha + itemBiases[item]
#     if item not in itemBiases:
#         return alpha + userBiases[user]
#     if user not in userBiases and item not in itemBiases:
#         return alpha
#     return alpha + userBiases[user] + itemBiases[item]

def prediction(user, item):
    if user not in userBiases:
        return alpha + itemBiases[item]
    if item not in itemBiases:
        return alpha + userBiases[user]
    if user not in userBiases and item not in itemBiases:
        return alpha
    return alpha + userBiases[user] + itemBiases[item]

In [2]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [3]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users1, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [4]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in userBookRatingsTrainSet]
    cost = MSE(predictions, labels)
#     print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [5]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(userBookRatingsTrainSet)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in userBookRatingsTrainSet:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - d[2]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users1] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

In [6]:
import scipy
import scipy.optimize

globalAverage = float(sum(allRatings)) / len(allRatings)
print(globalAverage)
userAverage = {}
bookAverage = {}
for u in userRatings:
    userAverage[u] = float(sum(userRatings[u])) / len(userRatings[u])
for b in bookRatings:
    bookAverage[b] = float(sum(bookRatings[b])) / len(bookRatings[b])
    
N = len(userBookRatingsTrainSet)
nUsers = len(userRatings)
nItems = len(bookRatings)
users1 = list(userRatings.keys())
items = list(bookRatings.keys())
alpha = globalAverage
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
labels = [x[2] for x in userBookRatingsTrainSet]
initialList = [alpha] + [0.0]*(nUsers+nItems)
theta = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems), derivative, args = (labels, 0.000008))
predictionsRatings = []
labels = [x[2] for x in validR]    
for l in validR:
    u,b = l[0], l[1]
    predictionsRatings.append(prediction(u, b))
print(MSE(predictionsRatings, labels))
print(alpha)

NameError: name 'allRatings' is not defined